In [6]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/srv/idp-radio-1'

In [7]:
import os 
import tensorflow as tf
from pathlib import Path

# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [8]:
relevant_columns = ['Edema', 'Atelectasis', 'Pneumonia', 'Lung Opacity', 'Consolidation']
chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                                                 classes=relevant_columns,
                                                 train_labels = "filtered_dependencies.csv",
                                                 batch_sizes = {"b": 32},
                                                 epoch_sizes = {"e": 5},
                                                 crop = {"C1": False},
                                                 split_seed = 6122156, )

Chestxray_BCE_E5_B32_C1_N5 could not be created
Chestxray_WBCE_E5_B32_C1_N5 could not be created
Chestxray_CWBCE_E5_B32_C1_N5 could not be created


In [9]:
chexpert_benchmarks.keys()

dict_keys(['BCE_E5_B32_C1_N5', 'WBCE_E5_B32_C1_N5', 'CWBCE_E5_B32_C1_N5'])

In [10]:
densenet_model = simple_architecture_experiment(chexpert_benchmarks["BCE_E5_B32_C1_N5"], DenseNet121, relevant_columns)

In [11]:
densenet_model.run()

Epoch 1/5
   1/1911 [..............................] - ETA: 0s - loss: 0.8480 - auc: 0.5437 - precision: 0.4123 - recall: 0.7833 - f2_score: 0.6638 - binary_accuracy: 0.5000 - learning rate: 0.0010WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


   2/1911 [..............................] - ETA: 5:37 - loss: 0.6302 - auc: 0.5967 - precision: 0.5203 - recall: 0.7264 - f2_score: 0.6731 - binary_accuracy: 0.6875 - learning rate: 0.0010WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1034s vs `on_train_batch_end` time: 0.2477s). Check your callbacks.


1911/1911 [==============================] - ETA: 0s - loss: 0.3687 - auc: 0.5788 - precision: 0.8792 - recall: 0.6338 - f2_score: 0.6713 - binary_accuracy: 0.8625 - learning rate: 0.0010INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.


INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.


...

INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.



Epoch 00005: val_loss did not improve from 0.35518
    7/19047 [..............................] - ETA: 4:49 - loss: 0.6655 - auc: 0.7283 - precision: 0.8571 - recall: 0.4000 - f2_score: 0.4478 - binary_accuracy: 0.7143  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


19047/19047 [==============================] - 310s 16ms/step - loss: 0.3696 - auc: 0.6292 - precision: 0.8973 - recall: 0.6274 - f2_score: 0.6676 - binary_accuracy: 0.8652
Expecting value: line 3587 column 5 (char 159439)


(<tensorflow.python.keras.callbacks.History at 0x7f7d44934a20>,
 {'report': '               precision    recall  f1-score   support\n\n        Edema       0.26      0.06      0.09      5201\n  Atelectasis       0.00      0.00      0.00      2747\n    Pneumonia       0.00      0.00      0.00       788\n Lung Opacity       0.91      1.00      0.96     17423\nConsolidation       0.00      0.00      0.00      2717\n\n    micro avg       0.88      0.61      0.72     28876\n    macro avg       0.23      0.21      0.21     28876\n weighted avg       0.60      0.61      0.59     28876\n  samples avg       0.90      0.70      0.76     28876\n',
  'metrics': {'loss': 0.36962762475013733,
   'auc': 0.6292253732681274,
   'precision': 0.8973305225372314,
   'recall': 0.6274414658546448,
   'f2_score': 0.6676000952720642,
   'binary_accuracy': 0.8652127385139465},
  'predictions': array([[0.67398286, 0.08370108, 0.00317831, 0.9694378 , 0.06858346],
         [0.41786012, 0.13407132, 0.0110496 , 0.95

In [12]:
from src.metrics.metrics import F2Score

dependencies = {
    'F2Score': F2Score
}
pretrained_model = tf.keras.models.load_model('models/DenseNet121_Chexpert_BCE_E5_B32_C1_N5/DenseNet121_Chexpert_BCE_E5_B32_C1_N5_20200721-112028.h5', custom_objects=dependencies)

In [ ]:
# freeze all layers except last one
for layer in pretrained_model.layers[:429]:
    layer.trainable = False
for layer in pretrained_model.layers:
    print(layer, layer.trainable)

In [16]:
unfiltered_chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                                                 classes=relevant_columns,
                                                 # train_labels = "filtered_dependencies.csv",
                                                 batch_sizes = {"b": 32},
                                                 epoch_sizes = {"e": 5},
                                                 crop = {"C1": False},
                                                 split_seed = 6122156, )
experiment = Experiment(unfiltered_chexpert_benchmarks["BCE_E5_B32_C1_N5"], pretrained_model, model_name='pretrained_DN_dependency')

Chestxray_BCE_E5_B32_C1_N5 could not be created
Chestxray_WBCE_E5_B32_C1_N5 could not be created
Chestxray_CWBCE_E5_B32_C1_N5 could not be created


In [17]:
experiment.run()

Epoch 1/5
   2/4447 [..............................] - ETA: 35:02 - loss: 0.6886 - auc: 0.6288 - precision: 0.8958 - recall: 0.6271 - f2_score: 0.6671 - binary_accuracy: 0.8649 - learning rate: 0.0010WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0545s vs `on_train_batch_end` time: 0.8909s). Check your callbacks.


4447/4447 [==============================] - ETA: 0s - loss: 0.3729 - auc: 0.6717 - precision: 0.6440 - recall: 0.4163 - f2_score: 0.4480 - binary_accuracy: 0.8349 - learning rate: 0.0010INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.


INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.



Epoch 00001: val_loss improved from inf to 0.37113, saving model to models/pretrained_DN_dependency/weights.01-0.37.hdf5
4447/4447 [==============================] - 271s 61ms/step - loss: 0.3729 - auc: 0.6717 - precision: 0.6440 - recall: 0.4163 - f2_score: 0.4480 - binary_accuracy: 0.8349 - learning rate: 0.0010 - val_loss: 0.3711 - val_auc: 0.6705 - val_precision: 0.5888 - val_recall: 0.3817 - val_f2_score: 0.4106 - val_binary_accuracy: 0.8325
Epoch 2/5
4447/4447 [==============================] - ETA: 0s - loss: 0.3700 - auc: 0.6725 - precision: 0.5911 - recall: 0.3737 - f2_score: 0.4034 - binary_accuracy: 0.8324 - learning rate: 0.0010INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.


INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.


...

INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.



Epoch 00005: val_loss did not improve from 0.36917
    5/44932 [..............................] - ETA: 9:33 - loss: 0.2955 - auc: 0.3167 - precision: 0.5000 - recall: 0.5000 - f2_score: 0.5000 - binary_accuracy: 0.8400               

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


44932/44932 [==============================] - 733s 16ms/step - loss: 0.3730 - auc: 0.6802 - precision: 0.5794 - recall: 0.4137 - f2_score: 0.4388 - binary_accuracy: 0.8305


(<tensorflow.python.keras.callbacks.History at 0x7f7bf89762b0>,
 {'report': '               precision    recall  f1-score   support\n\n        Edema       0.24      0.11      0.15     10461\n  Atelectasis       0.00      0.00      0.00      6912\n    Pneumonia       0.00      0.00      0.00      1225\n Lung Opacity       0.48      0.57      0.52     21324\nConsolidation       0.00      0.00      0.00      3063\n\n    micro avg       0.44      0.31      0.36     42985\n    macro avg       0.14      0.14      0.13     42985\n weighted avg       0.29      0.31      0.29     42985\n  samples avg       0.26      0.23      0.23     42985\n',
  'metrics': {'loss': 0.3730371296405792,
   'auc': 0.6802114844322205,
   'precision': 0.5793725848197937,
   'recall': 0.41374897956848145,
   'f2_score': 0.4388388991355896,
   'binary_accuracy': 0.8304959535598755},
  'predictions': array([[0.23357654, 0.14836957, 0.01322196, 0.5424773 , 0.04217318],
         [0.7421715 , 0.12319779, 0.02377194, 0.66

In [18]:
relevant_columns = ['Edema', 'Atelectasis', 'Pneumonia', 'Lung Opacity', 'Consolidation']
comp_chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                                                 name_suffix="_CTCOMP",
                                                 classes=relevant_columns,
                                                 # train_labels = "filtered_dependencies.csv",
                                                 batch_sizes = {"b": 32},
                                                 epoch_sizes = {"e": 5},
                                                 crop = {"C1": False},
                                                 split_seed = 6122156, )

Chestxray_BCE_E5_B32_C1_N5_CTCOMP could not be created
Chestxray_WBCE_E5_B32_C1_N5_CTCOMP could not be created
Chestxray_CWBCE_E5_B32_C1_N5_CTCOMP could not be created


In [19]:
comp_chexpert_benchmarks.keys()

dict_keys(['BCE_E5_B32_C1_N5_CTCOMP', 'WBCE_E5_B32_C1_N5_CTCOMP', 'CWBCE_E5_B32_C1_N5_CTCOMP'])

In [ ]:
densenet_model = simple_architecture_experiment(comp_chexpert_benchmarks["BCE_E5_B32_C1_N5_CTCOMP"], DenseNet121, relevant_columns)
densenet_model.run()

Epoch 1/5
   2/4447 [..............................] - ETA: 1:00:27 - loss: 0.8541 - auc: 0.6796 - precision: 0.5778 - recall: 0.4140 - f2_score: 0.4388 - binary_accuracy: 0.8301 - learning rate: 5.0000e-04WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1140s vs `on_train_batch_end` time: 1.5179s). Check your callbacks.


4447/4447 [==============================] - ETA: 0s - loss: 0.3548 - auc: 0.7198 - precision: 0.6244 - recall: 0.4311 - f2_score: 0.4596 - binary_accuracy: 0.8431 - learning rate: 5.0000e-04INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.


INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.



Epoch 00005: val_loss did not improve from 0.37542
    4/44932 [..............................] - ETA: 9:49 - loss: 0.1310 - auc: 0.2000 - precision: 1.0000 - recall: 1.0000 - f2_score: 1.0000 - binary_accuracy: 1.0000                

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


34340/44932 [=====================>........] - ETA: 3:00 - loss: 0.4004 - auc: 0.6674 - precision: 0.6530 - recall: 0.1881 - f2_score: 0.2193 - binary_accuracy: 0.8261